In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.interpolate import griddata
from PIL import Image

In [2]:
# Load in array of 201 images, create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(index=np.arange(201))
my_dpi = 192

r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

# Create convert function to use to convert im_array (203x203) back to raw values
phimax, phimin = np.max(phi), np.min(phi)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin

for i in range(201):
    arr = phi[:,:,i]
    values = arr.flatten()
    
    # Interpolate the data to the Cartesian grid
    grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')

    # Plot image (raw automatically converted to grayscale), save as PNG
    plt.figure(figsize=(264/my_dpi, 264/my_dpi), dpi=my_dpi)
    plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
    plt.axis("off")

    # Save as PNG then reopen. Then save as JPG's of varying quality (for loop). Convert back to np array.
    plt.savefig('2.2.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('2.2.png')
    im = im.convert('RGB') # need to convert from RGBA to RGB, otherwise saving as JPG doesn't work
    
    for q in [95, 75, 50, 25]:
        title = '2.2.' + str(q) + '.jpg'
        im.save(title, quality=q)
        im_jpg = Image.open(title)
        im_array = np.asarray(im_jpg)

        # Loop through im_array and replace RGB by raw in copy (203x203)
        copy = np.zeros_like(grid_z)
        for row_index, row in enumerate(im_array):
            for col_index, col in enumerate(row):
                copy[row_index][col_index] = convertRGBToRaw(col[0])
    
        # Get output_array from copy by computing nearest pixels
        output_array = np.zeros((101, 227))
        for rad in range(101):
            for ang in range(227):
                # Polar coordinates
                radius = r[rad]
                angle = theta[ang]
                
                # Convert to Cartesian coordinates
                x_cart = radius * np.cos(angle)
                y_cart = radius * np.sin(angle)
                
                # Find the nearest pixel in the Cartesian grid
                x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
                y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
                
                # Get the raw values from copy and put into output array
                output_array[rad, ang] = copy[y_nearest, x_nearest]
                
        # Compute compression ratio and RMSE's, add to dataframe
        in_imsize = os.stat('2.2.png').st_size
        out_imsize = os.stat(title).st_size
        crcol = 'CR2.2.' + str(q)
        df.at[i, crcol] = in_imsize/out_imsize

        diff = output_array - arr
        rmse = np.sqrt(np.mean(diff**2))
        rmsecol = 'RMSE2.2.' + str(q)
        df.at[i, rmsecol] = rmse
    
df.to_pickle("analysis2.2.pkl")
df

,CR2.2.95,RMSE2.2.95,CR2.2.75,RMSE2.2.75,CR2.2.50,RMSE2.2.50,CR2.2.25,RMSE2.2.25
0,2.731508,0.453644,5.593847,0.453540,7.014160,0.453204,7.897196,0.454432
1,2.894597,0.316969,5.970850,0.316869,7.550304,0.316868,9.082420,0.317324
2,2.682558,0.454805,5.481357,0.454845,6.973504,0.454646,8.468085,0.455200
3,2.804874,0.533241,5.962810,0.533309,7.452920,0.533359,9.146270,0.533505
4,2.809298,0.340551,5.937298,0.340568,7.513292,0.340626,9.217260,0.340867
...,...,...,...,...,...,...,...,...
196,2.911860,0.141710,5.910355,0.141837,7.215094,0.141665,8.588433,0.141244
197,2.918494,0.133904,5.838121,0.133890,7.178520,0.134213,8.496610,0.133553
198,2.824143,0.086159,5.736336,0.086290,7.013176,0.086387,8.396056,0.086236
199,2.814121,0.071590,5.766874,0.071491,7.076154,0.072086,8.418460,0.072248


In [5]:
df.isnull().sum().sum()

0

In [26]:
# Testing single images
i = 201

phi = np.load("Phi_201snap.npy")
r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

arr = phi[:,:,i]
values = arr.flatten()
    
# Interpolate the data to the Cartesian grid
grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')
    
# Plot and save image
plt.figure(figsize=(264/my_dpi, 264/my_dpi), dpi=my_dpi)
plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
plt.axis("off")
plt.savefig('analysis2.2.jpg', bbox_inches='tight', pad_inches=0)
im = Image.open('analysis2.2.jpg')
im_array = np.asarray(im)

# Create convert function to use to convert im_array back to raw values
phimax, phimin = np.max(grid_z), np.min(grid_z)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin

# Loop through im_array and replace RGB by raw in copy
copy = grid_z.copy()
for i, row in enumerate(im_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])

output_array = np.zeros((101, 227))
for rad in range(101):
    for ang in range(227):
        # Polar coordinates
        radius = r[rad]
        angle = theta[ang]
            
        # Convert to Cartesian coordinates
        x_cart = radius * np.cos(angle)
        y_cart = radius * np.sin(angle)
            
        # Find the nearest pixel in the Cartesian grid
        x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
        y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
            
        # Get the raw values from copy and put into output array
        output_array[rad, ang] = copy[y_nearest, x_nearest]
        
# Compute norm, add to dataframe
diff_array = output_array - arr
norm = np.linalg.norm(diff_array)
norm

IndexError: index 201 is out of bounds for axis 2 with size 201